<a href="https://colab.research.google.com/github/AndreMartins21/Teste_Nivelamento_IntuitiveCare/blob/main/Teste2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TESTE 2
## <> ETAPAS:
  - Extrair do pdf do teste 1 os dados dos Quadros 30,31,32 (Tabela de categoria do Padrão TISS);
  - Salvar dados dessas tabelas em csvs;
  - Zipar todos os csvs num arquivo "Teste_Intuitive_Care_{seu_nome}.zip".

In [2]:
!pip install tabula-py

In [3]:
import tabula
import pandas as pd
import zipfile

def obter_tabelas():
  '''
  Retornar todos os quadros 30, 31 e 32 do PDF (P.S: Esse está salvo no Github)
  '''

  pdf = "https://github.com/AndreMartins21/Teste_Nivelamento_IntuitiveCare/blob/main/Arquivos/Padr%C3%A3o_TISS_Componente_Organizacional_202103.pdf?raw=" 

  tabelas = tabula.read_pdf(pdf, pages = [79, 80, 81, 82, 83, 84, 85])
  
  return tabelas

tabelas = obter_tabelas()

In [4]:
def quadro_30():
  '''
  Configurando o dataframe n° 30 para converter em csv
  '''

  df_30 = tabelas[0]
  
  # SE 1° VEZ RODANDO..
  if df_30.columns[0] == "Tabela de Tipo do Demandante":

    df_30["Código"] = list(range(6))
    df_30.drop(0, axis = 0, inplace = True) # Dropando a 1° linha
    df_30.rename(columns = {"Tabela de Tipo do Demandante": "Descrição da Categoria"}, inplace = True) # Renomeando 2° coluna
    df_30["Descrição da Categoria"] = df_30["Descrição da Categoria"].apply(lambda x: x[1:]) # Editando os valores da 2° coluna
  
  return df_30[["Código", "Descrição da Categoria"]]

quadro_30()

,Código,Descrição da Categoria
1,1,Operadora
2,2,Prestador de serviço
3,3,Consumidor
4,4,Gestor
5,5,ANS


In [5]:
# Tabelas[1:7] representam todos as tabelas que formam o quadro 31

tabelas[6].tail()

,146,Mensagem de comunicação de internação ou alta do beneficiário\r(comunicacaoInternacao)
17,164,Guia de resumo de internação
18,165,Guia de serviços profissionais/serviço auxilia...
19,166,Guia de solicitação de internação
20,167,Guia de solicitação de prorrogação de internaç...
21,168,Guia de tratamento odontológico


Pode-se perceber acima que as colunas do DF eram para ser, na realidade, linhas. Logo, para resolver esse impasse, utilizei da seguinte estratégia:


In [6]:
def organizar_31(i=1):
  line_col = list(tabelas[i].columns) # Salvar o nome atual das colunas
  col = ["Código", "Descrição da Categoria"] 
  
  # Se as colunas já estiverem no padrão, retornar o df normal
  if line_col == col: 
    return tabelas[i]
  
  df_col = pd.DataFrame({
      col[0]: [line_col[0]],
      col[1]: [line_col[1]]
  })

  tabelas[i].rename(columns={line_col[0]: col[0], line_col[1]: col[1]}, inplace = True) # Renomeando as colunas

  # O 1° quadro apresenta colunas diferentes do restante
  if i != 1:
    tabelas[i] = pd.concat([df_col, tabelas[i]], ignore_index = True)  
  # Remover a linha 0
  if tabelas[i][0:1].values.any():
    tabelas[i].drop(0, axis = 0, inplace = True)
  return tabelas[i]

organizar_31(i=6).tail()

,Código,Descrição da Categoria
18,164,Guia de resumo de internação
19,165,Guia de serviços profissionais/serviço auxilia...
20,166,Guia de solicitação de internação
21,167,Guia de solicitação de prorrogação de internaç...
22,168,Guia de tratamento odontológico


In [7]:
def quadro_31():
  '''
  Função responsável por montar o df 31, mesclando todos os quadros
  '''

  df_31 = pd.concat([organizar_31(i) for i in range(1, 7)], ignore_index = True) # Concatenar todos os quadros 31 
  
  df_31["Descrição da Categoria"] = df_31["Descrição da Categoria"].str.replace("\r", " ")
  
  return df_31

quadro_31()

,Código,Descrição da Categoria
0,1,Componente Organizacional
1,2,Componente de Conteúdo e Estrutura
2,3,Componente de Representação de Conceitos em Saúde
3,5,Componente de Segurança e Privacidade
4,18,"Terminologia de diárias, taxas e gases medicinais"
...,...,...
121,164,Guia de resumo de internação
122,165,Guia de serviços profissionais/serviço auxilia...
123,166,Guia de solicitação de internação
124,167,Guia de solicitação de prorrogação de internaç...


In [8]:
def quadro_32():
  '''
  Função que irá organizar o df 32
  '''
  df_32 = tabelas[7]

  # SE 1° VEZ RODANDO..
  if df_32.columns[0] == "Tabela de Tipo de Solicitação":
    df_32["Código"] = list(range(4))
    
    df_32.rename(columns= {"Tabela de Tipo de Solicitação": "Descrição da Categoria"}, inplace= True) # Renomeando as colunas
    df_32.drop(0, axis = 0, inplace = True)
    df_32["Descrição da Categoria"] = df_32["Descrição da Categoria"].apply(lambda x: x[1:])
    
  return df_32[["Código", "Descrição da Categoria"]]

quadro_32()

,Código,Descrição da Categoria
1,1,Alteração
2,2,Inclusão
3,3,Exclusão


**SALVANDO TODOS DATA_FRAMES EM CSV E ZIPANDO**

In [9]:
dfs = [quadro_30(), quadro_31(), quadro_32()]
for cont in range(3):
  dfs[cont].to_csv(f"Quadro3{cont}.csv", encoding = "utf-8")
  print(f"O quadro 3{cont} foi convertido em CSV com sucesso!")

O quadro 30 foi convertido em CSV com sucesso!
O quadro 31 foi convertido em CSV com sucesso!
O quadro 32 foi convertido em CSV com sucesso!


In [10]:
def zipar():
  nome = "Teste_Intuitive_Care_Andre_Martins.zip"
  z = zipfile.ZipFile(nome, 'w', zipfile.ZIP_DEFLATED)
  for i in range(3):
    z.write(f'Quadro3{i}.csv')
  z.close()
  return "CSV's zipados com êxito!"
zipar()

"CSV's zipados com êxito!"